## Рекомендательные системы: предсказание оценки

Рассмотрим задачу предсказания оценки, которую пользователь поставит фильму. Особенность этой задачи в том, что объекты выборки описываются категориальными признаками, принимающими большое число значений (например: идентификатор пользователя, идентификатор фильма, тэги, киноперсоны).

Данные: [MovieLens + IMDb/Rotten Tomatoes](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip) ([описание](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-readme.txt)). Набор содержит данные о предпочтениях пользователей сервиса рекомендации кинофильмов [MovieLens](http://www.movielens.org/). Пользовательские оценки для фильмов принимают значения в интервале от 0.5 до 5.0, они записаны в файле *user_ratedmovies.dat* (а также в *user_ratedmovies-timestamps.dat*,  где для каждой оценки записана дата и время в формате timestamp), остальные файлы содержат дополнительную информацию о фильмах, которую можно использовать как признаки. Заметьте: кроме оценок (и тегов), про пользователя ничего не известно.

Задача: построить модель, предсказывающую оценку пользователя фильму, который он еще не смотрел.

Метрика качества: будем считать, что пользователю сервиса доступен блок рекомендаций, который может содержать рекомендации не более чем 5 фильмов.
Выберем некоторого пользователя $u$ и обозначим известные для него рейтинги за $R^u$. В качестве тестовых рейтингов $R^u_{test}$ для этого пользователя рассмотрим 5 рейтингов, поставленные последними по времени, в качестве валидационных $R^u_{val}$ — предпоследние 5 рейтингов. Остальные известные рейтинги этого пользователя будут составлять обучающую выборку $R^u_{train}$.
Для подбора гиперпараметров в рамках данного задания будем использовать валидационную выборку, предварительно обучив модель на обучающей выборке, а для финальной оценки качества — тестовую выборку, предварительно обучив модель на обучающей и валидационной выборках.

**1. (1 балл)** Загрузите данные и сформируйте 3 разреженные матрицы пользователи—фильмы для обучающих, валидационных и тестовых рейтингов пользователей соответственно, где в каждой ячейке стоит рейтинг, если он известен, или ноль, если неизвестен.

In [3]:
import numpy as np
import pandas as pd
from scipy import sparse
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
user_rate = pd.read_csv('user_ratedmovies-timestamps.dat', sep='\t')
user_rate.head()

,userID,movieID,rating,timestamp
0,75,3,1.0,1162160236000
1,75,32,4.5,1162160624000
2,75,110,4.0,1162161008000
3,75,160,2.0,1162160212000
4,75,163,4.0,1162160970000


In [5]:
test=pd.DataFrame(columns=[user_rate.columns])
val=pd.DataFrame(columns=[user_rate.columns])
train=pd.DataFrame(columns=[user_rate.columns])

In [6]:
for user in np.unique(user_rate.userID):
    current = user_rate[user_rate.userID == user].sort_values(by='timestamp', ascending=False)
    
    if len(current.movieID) > 5:
        test = test.append(current[:5])
    if len(current.movieID) - 5 > 5:
        val = val.append(current[5:10])
    if len(current.movieID) - 10 > 0:
        train = train.append(current[10:])

*Pivot table comment*

In [7]:
test.pivot_table(index='userID', values='rating',columns='movieID',fill_value=0).head()

movieID,1,2,3,4,5,6,7,10,11,16,...,64976,64983,64986,64990,64997,65011,65037,65126,65130,65133
userID,,,,,,,,,,,,,,,,,,,,,
75,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0
78,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0
127,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0
170,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0
175,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,0.0,...,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0


In [8]:
test = sparse.csr_matrix(test.pivot_table(index='userID', values='rating',columns='movieID',fill_value=0).values)
val = sparse.csr_matrix(val.pivot_table(index='userID', values='rating',columns='movieID',fill_value=0).values)
train = sparse.csr_matrix(train.pivot_table(index='userID', values='rating',columns='movieID',fill_value=0).values)

Качество рекомендаций: будем использовать метрики RMSE@k и nDCG@k для $k=5$, описанные ниже.

#### RMSE@k

Поскольку нас интересуют лишь фильмы, попавшие в блок рекомендаций, качество работы модели можно оценивать при помощи RMSE на $k$ фильмах с наибольшим предсказанным рейтингом, где $k$ — размер блока рекомендаций. Отсортируем предсказанные моделью рейтинги $\hat{r}_{ui}$ в порядке убывания и обозначим $i$-ый элемент в полученной последовательности за $\hat{r}_{u(i)},$ а соответствующее этому фильму истинное значение рейтинга — за $r_{u(i)}$. Тогда RMSE@k:

$$ \text{RMSE@k}(u) = \sqrt{ \frac{1}{k} \sum_{i=1}^k (r_{u(i)} - \hat{r}_{u(i)})^2 },$$
$$ \text{RMSE@k} = \frac{1}{|U|} \sum_{u \in U} \text{RMSE@k}(u),$$
где $U$ — множество пользователей. При вычислении данной метрики все неизвестные оценки будем полагать равными 0.

#### nDCG@k

Также можно использовать метрику качества ранжирования. Для этого для каждого пользователя $u$ предскажем оценку для всех фильмов из $R^u_{test}$ и отсортируем эти фильмы по убыванию предсказанного рейтинга. Ожидается, что хороший алгоритм должен выдать релевантные фильмы вверху списка. Отсортируем предсказанные моделью рейтинги $\hat{r}_{ui}$ в порядке убывания и обозначим $i$-ый элемент в полученной последовательности за $\hat{r}_{u(i)},$ а соответствующее этому фильму истинное значение рейтинга — за $r_{u(i)}.$

Тогда nDCG@k :

$$\text{DCG@k}(u) = \sum_{i=1}^k g(r_{u(i)}) d(i),$$
$$\text{nDCG@k}(u) = \frac{\text{DCG@k}(u)}{\max \text{DCG@k}(u)},$$
$$\text{nDCG@k} = \frac{1}{|U|} \sum_{u \in U} \text{nDCG@k}(u),$$
где $g(r)$ — функция полезности фильма, а  $d(i)$ — штраф за позицию.

Положим $g(r) = 2^r-1, \, d(i) = \frac{1}{\log_2 (i+1)}.$ При вычислении данной метрики все неизвестные оценки будем полагать равными 0.

**2. (2 балла)** Реализуйте функции rmse_score и ndcg_score, вычисляющие значения описанных выше метрик. Каждая из функций в качестве параметров должна принимать:
 * y_true — матрицу тестовых рейтингов (сформированную аналогично матрице тестовых рейтингов из предыдущего пункта; функция должна корректно работать и для разреженных, и для плотных матриц);
 * y_predicted — матрицу предсказаний модели в аналогичном формате (функция должна корректно работать и для разреженных, и для плотных матриц);
 * k — параметр $k$ в определениях метрик.

In [49]:
def rmse_score(y_true, y_predicted, k=5):
    if type(y_true) == sparse.csr.csr_matrix:
        y_true_ = y_true.toarray()
    else:
        y_true_ = y_true
    if type(y_predicted) == sparse.csr.csr_matrix:
        y_predicted_ = y_predicted.toarray()
    else:
        y_predicted_ = y_predicted
    rmse = 0
    for i, u in enumerate(y_predicted_):
        sort_arr = np.argsort(-u)
        rmse += np.sqrt(np.sum((u[sort_arr[:k]] - y_true_[i, sort_arr[:k]])**2)*1/k)
    
    return rmse/y_true.shape[0]
def ndcg_score(y_true, y_predicted, k=5):
    if type(y_true) == sparse.csr.csr_matrix:
        y_true_ = y_true.toarray()
    else:
        y_true_ = y_true
    if type(y_predicted) == sparse.csr.csr_matrix:
        y_predicted_ = y_predicted.toarray()
    else:
        y_predicted_ = y_predicted
    ndcg = []
    for i, u in enumerate(y_true_):
        sort_arr = np.argsort(-y_predicted_[i])
        dcg = sum((2**u[sort_arr][:5] - 1)*1/np.log2(i + 2))
        sort_arr = np.argsort(-y_true_[i])
        maxdcg = sum((2**u[sort_arr][:5] - 1)*1/np.log2(i + 2))
        ndcg.append(dcg/maxdcg)
    return sum(ndcg)/y_true.shape[0]

**3. (1 балл)** Постройте рекомендации на основе **most popular** метода, при котором предсказанный рейтинг для некоторого фильма $i$ одинаков для всех пользователей и совпадает со средним значением рейтинга по всем пользователям, оценившим этот фильм, и вычислите значения метрик RMSE@5 и nDCG@5 для тестовой матрицы из п. 1.

In [90]:
most_pop = np.zeros((test.shape[0], test.shape[1]))

for i in range(most_pop.shape[1]):
    most_pop[:, i] = sum(test[:, i].toarray())[0]/(test[:, i].count_nonzero())
    

In [92]:
rmse_score(test, most_pop, k=5)

4.998501095944154

In [93]:
ndcg_score(test, most_pop, k=5)

0.00093196744047250755

**4. (1 балл)** Реализуйте построение рекомендаций путём разложения матрицы рейтингов с помощью [разреженного SVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) (в предположении, что неизвестные рейтинги заменяются на нули) и последующего её восстановления и постройте график зависимости метрик RMSE@5 и nDCG@5 от значения ранга разложения на валидационной выборке (рассмотрите как минимум 10 различных значений ранга разложения)

In [22]:
from sklearn.decomposition import TruncatedSVD

In [123]:
svd = TruncatedSVD(n_components=100, random_state=42)

In [124]:
tmp = svd.fit_transform(test)

In [125]:
svd_res = svd.inverse_transform(tmp)

In [126]:
svd_res

array([[ 0.15793004, -0.03722675, -0.00130783, ...,  0.00329775,
         0.00122571, -0.00535085],
       [-0.01655634,  0.15080364,  0.01111472, ..., -0.00471254,
         0.00295217,  0.01499056],
       [-0.00634457, -0.00125025, -0.00146639, ...,  0.0008121 ,
         0.00217583,  0.00253884],
       ..., 
       [ 0.01053898,  0.01063534,  0.00673362, ..., -0.00447147,
        -0.00161573, -0.00348121],
       [ 0.04196959,  0.00118117, -0.02768266, ...,  0.00459425,
         0.00103762, -0.00399826],
       [ 0.04883649, -0.00702768, -0.00102678, ..., -0.00082063,
         0.00034337,  0.00737704]])

In [127]:
rmse_score(test, svd_res, k=5)

1.207212060621913

In [128]:
ndcg_score(test, svd_res, k=5)

0.43648980773560697

**another approach**

In [23]:
from scipy.sparse.linalg import svds

In [118]:
U, sigma, Vt = svds(test, k = 100)

In [119]:
sigma = np.diag(sigma)

In [120]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [121]:
rmse_score(test, all_user_predicted_ratings, k=5)

1.1312537312740119

In [122]:
ndcg_score(test, all_user_predicted_ratings, k=5)

0.4179330390034604

**5. (3 балла)** Постройте рекомендации на основе user-based коллаборативной фильтрации. 
Предсказание модели $\hat{r}_{ui}$ вычисляйте по следующей формуле:
$$\hat{r}_{ui} = \bar{r}_{u} + \frac{\sum_{v \in U(u)} w_{uv} (r_{vi} - \bar{r}_v)}{\sum_{v \in U(u)} w_{uv}},$$
где $\bar{r}_u$ — средний ретинг пользователя $u$, $w_{uv}$ — мера сходства пользователей $u$ и $v$, $U(u) = \{ v \in U \, | \, w_{uv} > \alpha\}$ — коллаборация пользователя $u$. 

Значение параметра $\alpha$ возьмите равным 0.9.

Вычислите значения метрик RMSE@5 и nDCG@5 на тестовой выборке.

In [14]:
from tqdm import tqdm, tqdm_notebook
import sklearn.preprocessing as pp

In [72]:
def cosine(mat):
    col_normed_mat = pp.normalize(mat.T.tocsc(), axis=0)
    return col_normed_mat.T * col_normed_mat

def pirson(a, b):
    a_ = a - np.mean(a)
    b_ = b - np.mean(b)
    return np.sum(a_*b_, axis=1)/(np.linalg.norm(a_, ord=2)*np.linalg.norm(b_, ord=2))
# requires corrections
def user_based(x, measure, alfa=0.9):
    if type(x) == sparse.csr.csr_matrix:
        x_ = x.toarray()
    else:
        x_ = x
    r_ = np.zeros((x.shape[0], x.shape[1]))
    for i, u in tqdm_notebook(enumerate(x_)):
        v = np.delete(x_, i, axis=0)
        r_u = np.mean(u)*np.ones(r_[i].shape)
        w_uv = measure(u, v)
        v = v[np.where(w_uv > alfa)]
        print(w_uv)
        w_uv = w_uv[w_uv > alfa].reshape(-1,1)
        
        if np.count_nonzero(w_uv) == 0:
            sum_w = 1.0
        else:
            sum_w = np.sum(w_uv)
            
        r_[i] = (r_u + np.sum(w_uv*(v - np.mean(v, axis=1).reshape(-1, 1)), axis=0)/sum_w)

    return r_

In [54]:
cosine(train).toarray()

array([[ 1.        ,  0.06854802,  0.        , ...,  0.12209127,
         0.0444811 ,  0.06484208],
       [ 0.06854802,  1.        ,  0.        , ...,  0.20276096,
         0.12704347,  0.27019313],
       [ 0.        ,  0.        ,  1.        , ...,  0.04574839,
         0.        ,  0.        ],
       ..., 
       [ 0.12209127,  0.20276096,  0.04574839, ...,  1.        ,
         0.18101983,  0.14529108],
       [ 0.0444811 ,  0.12704347,  0.        , ...,  0.18101983,
         1.        ,  0.1602749 ],
       [ 0.06484208,  0.27019313,  0.        , ...,  0.14529108,
         0.1602749 ,  1.        ]])

In [33]:
train_pred=user_based(train, cosine)

[ 0.00333499  0.          0.00264068 ...,  0.00542932  0.00080814
  0.00188945]
[ 0.00089123  0.          0.00235888 ...,  0.00901881  0.00230869
  0.00787511]
[ 0.         0.         0.        ...,  0.0020344  0.         0.       ]
[ 0.00238799  0.00798228  0.         ...,  0.00647949  0.00444146
  0.00727206]


KeyboardInterrupt: 

**6. (1 балл)** Какой метод оказался лучше по каким метрикам? Почему?

**7. (1 балл)** Приведите достоинства и недостатки используемых метрик. Какие еще метрики можно было бы использовать для решения задачи? Приведите примеры других постановок задачи, как в этом случае можно было бы оценить качество?